In [12]:
words = open('names.txt', 'r').read().splitlines()


In [13]:
words[:10]


['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [14]:
len(words)


32033

In [15]:
import torch

In [82]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
    

In [96]:
itos

{1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z',
 0: '.'}

In [32]:
N = torch.zeros((27*27+27, 27), dtype=torch.int32)
N.shape

torch.Size([729, 27])

In [122]:
xs, ys = [], []

for word in words[:1]:
    word = '.' + word + '.'
    tuples = [([word[i],word[i+1]], word[i+2]) for i in range(len(word)-2)]
    for [x1,x2],y in tuples:
        ix1 = stoi[x1]
        ix2 = stoi[x2]
        iy = stoi[y]
        xs.append((ix1,ix2))
        ys.append(iy)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

In [124]:
xs

tensor([[ 0,  5],
        [ 5, 13],
        [13, 13],
        [13,  1]])

In [112]:
ys.shape

torch.Size([4])

In [134]:
import torch.nn.functional as F
xenc = F.one_hot(xs, num_classes=27*27).float()


In [133]:
xenc.shape

torch.Size([4, 2, 729])

In [140]:
xenc[0,0]

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

In [141]:
xenc.dtype


torch.float32

In [132]:
# randomly initialize 27 neurons' weights. each neuron receives 27 inputs
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27*27, 27), generator=g)

In [131]:
W.shape

torch.Size([729, 729])

In [128]:
xenc = F.one_hot(xs, num_classes=27*27).float() # input to the network: one-hot encoding
logits = xenc @ W # predict log-counts
counts = logits.exp() # counts, equivalent to N
probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
# btw: the last 2 lines here are together called a 'softmax'

In [144]:
probs.shape


torch.Size([4, 2, 27])

In [145]:
probs

tensor([[[0.7493, 0.1518, 0.4146, 0.1034, 0.1079, 0.3780, 0.2910, 0.4971,
          0.6684, 0.5974, 0.1385, 0.6347, 0.5318, 0.3937, 0.8681, 0.9151,
          0.7479, 0.4404, 0.2797, 0.2665, 0.4991, 0.3419, 0.0759, 0.9133,
          0.5883, 0.9903, 0.8152],
         [0.2507, 0.8482, 0.5854, 0.8966, 0.8921, 0.6220, 0.7090, 0.5029,
          0.3316, 0.4026, 0.8615, 0.3653, 0.4682, 0.6063, 0.1319, 0.0849,
          0.2521, 0.5596, 0.7203, 0.7335, 0.5009, 0.6581, 0.9241, 0.0867,
          0.4117, 0.0097, 0.1848]],

        [[0.5692, 0.6058, 0.4215, 0.6902, 0.8075, 0.6719, 0.3360, 0.6563,
          0.1012, 0.7228, 0.9303, 0.2916, 0.5543, 0.4902, 0.5118, 0.7994,
          0.4139, 0.3462, 0.6671, 0.8398, 0.7688, 0.8446, 0.8430, 0.1581,
          0.1221, 0.0614, 0.8415],
         [0.4308, 0.3942, 0.5785, 0.3098, 0.1925, 0.3281, 0.6640, 0.3437,
          0.8988, 0.2772, 0.0697, 0.7084, 0.4457, 0.5098, 0.4882, 0.2006,
          0.5861, 0.6538, 0.3329, 0.1602, 0.2312, 0.1554, 0.1570, 0.8419,
     

In [153]:
nlls = torch.zeros(5)
for i in range(4):
    for j in range(2):
  # i-th bigram:
        x = xs[i][j].item() # input character index
        y = ys[i].item() # label character index
        print('--------')
        print(f'bigram example {i+1}: {itos[x]}{itos[y]} (indexes {x},{y})')
        print('input to the neural net:', x)
        print('output probabilities from the neural net:', probs[i])
        print('label (actual next character):', y)
        p = probs[i,j,y]
        print('probability assigned by the net to the the correct character:', p.item())
        logp = torch.log(p)
        print('log likelihood:', logp.item())
        nll = -logp
        print('negative log likelihood:', nll.item())
        nlls[i] = nll

print('=========')
print('average negative log likelihood, i.e. loss =', nlls.mean().item())

--------
bigram example 1: .m (indexes 0,13)
input to the neural net: 0
output probabilities from the neural net: tensor([[0.7493, 0.1518, 0.4146, 0.1034, 0.1079, 0.3780, 0.2910, 0.4971, 0.6684,
         0.5974, 0.1385, 0.6347, 0.5318, 0.3937, 0.8681, 0.9151, 0.7479, 0.4404,
         0.2797, 0.2665, 0.4991, 0.3419, 0.0759, 0.9133, 0.5883, 0.9903, 0.8152],
        [0.2507, 0.8482, 0.5854, 0.8966, 0.8921, 0.6220, 0.7090, 0.5029, 0.3316,
         0.4026, 0.8615, 0.3653, 0.4682, 0.6063, 0.1319, 0.0849, 0.2521, 0.5596,
         0.7203, 0.7335, 0.5009, 0.6581, 0.9241, 0.0867, 0.4117, 0.0097, 0.1848]])
label (actual next character): 13
probability assigned by the net to the the correct character: 0.3937307298183441
log likelihood: -0.9320880174636841
negative log likelihood: 0.9320880174636841
--------
bigram example 1: em (indexes 5,13)
input to the neural net: 5
output probabilities from the neural net: tensor([[0.7493, 0.1518, 0.4146, 0.1034, 0.1079, 0.3780, 0.2910, 0.4971, 0.6684,
       

In [ ]:
# ALL TOGETHER NOW!!

In [339]:
xs, ys = [], []
for word in words:
    word = '.' + word + '.'
    tuples = [([word[i],word[i+1]], word[i+2]) for i in range(len(word)-2)]
    for [x1,x2],y in tuples:
        ix1 = stoi[x1]
        ix2 = stoi[x2]
        iy = stoi[y]
        xs.append((ix1,ix2))
        ys.append(iy)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad=True)

In [340]:
ys.shape

torch.Size([196113])

In [341]:
W.shape

torch.Size([27, 27])

In [ ]:
xs.shape

torch.Size([196113, 2])

In [373]:
num = len(xs)
for k in range(300):
    xenc = F.one_hot(xs, num_classes=27).float() # input to the network: one-hot encoding
    #xenc = F.one_hot(xs, num_classes=27*27).float() # input to the network: one-hot encoding
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    probs = counts / counts.sum(-1, keepdims=True) # probabilities for next character
    valid_indices = (ys >= 0) & (ys < 27)
    loss = -probs[torch.arange(num)[valid_indices], :, ys[valid_indices]].log().mean() + 0.01*(W**2).mean()
    print(loss.item())
    
    # backward pass
    W.grad = None # set to zero the gradient
    loss.backward()

    # update
    W.data += -50 * W.grad

2.6039958000183105
2.59623384475708
2.593493700027466
2.5934407711029053
2.593430995941162
2.5934269428253174
2.593424081802368
2.593421697616577
2.5934197902679443
2.5934181213378906
2.593416690826416
2.5934150218963623
2.5934133529663086
2.593412399291992
2.5934109687805176
2.593409538269043
2.5934083461761475
2.593407154083252
2.5934059619903564
2.593404769897461
2.5934035778045654
2.59340238571167
2.5934009552001953
2.5933997631073
2.5933985710144043
2.593397378921509
2.5933966636657715
2.593395471572876
2.5933942794799805
2.593392848968506
2.5933918952941895
2.593390941619873
2.5933897495269775
2.593388557434082
2.5933878421783447
2.593386650085449
2.593385696411133
2.5933847427368164
2.593383550643921
2.5933825969696045
2.593381404876709
2.5933806896209717
2.593379259109497
2.5933783054351807
2.5933775901794434
2.593376636505127
2.5933754444122314
2.593374490737915
2.5933735370635986
2.593372344970703
2.593371629714966
2.5933709144592285
2.593369483947754
2.5933687686920166
2.593

In [370]:
# finally, sample from the 'neural net' model
g = torch.Generator().manual_seed(2147483647)
for i in range(100):
    out = []
    ix = [stoi['.'], stoi['.']]
    while True:
        xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
        logits = xenc @ W # predict log-counts
        counts = logits.exp() # counts, equivalent to N
        p = counts / counts.sum(-1, keepdims=True) # probabilities for next character
        ix = torch.multinomial(p[-1], num_samples=1, replacement=True, generator=g)[0].item()
        out.append(itos[ix])
        input_seq = torch.cat((input_seq[1:], torch.tensor([ix], dtype=torch.long)))
        if ix == 0:
            break
        ix = [ix, stoi['.']] # shift the sequence by one character
    print(''.join(out[:-1]))

mn
univrya
ri
rd
iuram
anegachainalmcye
au
ey
iklen
ni
a
eikavr
hhnte
usi
atrynalmlynalaraiowo
ri
wele
endmielayayqzahihynaylortrakriyniamchraicrcaleth
o
s
ydm
aon
i
e
oi
aahslaisoa
ieeayaalalasaynl
uaker
on
h
alelqokorldeimaerleljdsahzllelolnauleyan
eha
ariosruvnwcyn
usia
ih
ha
as
uthailoeoxyn
ihvanain
tihniln
arno
madiasnn
rih
e
ac
ubiun
av
anolama
ope
a
yndmu
yely
i
ianeteo
uahhaio
h
maletfarahaajaaclaien
are
al
o
avemanin
yidlaeto
el
a
ureen
ambyaliesoukpnky
iladamoeastz
ae
hlan
e
e
atn
aotsmar
r
alya
axzee
otyanichsa
ontinteri
a
zitrian
ionadaoetlleioraxde
daiaan
ylisel
his
au
veviaamreynsaiy
aiaayleblaaxtamrbrwdeln
manayn
aen
ai
rhfars
a
a
oo
ainy
eha
yahadnn
oeiyneyarllninir
n
ariayaisenyxfxghetma
